In [1]:
import sys
sys.path.append('/usr/project/xtmp/sk787/variable_imp_matching/')
sys.path.insert(0, '/usr/project/xtmp/sk787/variable_imp_matching/')

import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from datagen.dgp_df import dgp_df

from confseq.betting import betting_ci
from sklearn.preprocessing import minmax_scale
from sklearn.ensemble import RandomForestRegressor

import warnings

In [ ]:
from Experiments.variance.randomization_coverage_exp import knn_match
from datagen.dgp import dgp_linear, dgp_lihua

In [ ]:
import importlib
importlib.reload(sys.modules['Experiments.variance.randomization_coverage_exp'])
from Experiments.variance.randomization_coverage_exp import knn_match

In [ ]:
n_train = 1000
n_est = 5000
n_query = 500
train_query_seed = 42
xmin = -5
xmax = 5
n_imp = 2
n_unimp = 0
heteroskedastic = 0
corr = False

k = 16 * int(np.sqrt(n_est))

gen_linear_dgp = dgp_lihua(xmin = xmin, xmax = xmax, n_imp = n_imp, n_unimp=n_unimp, heteroskedastic=heteroskedastic, corr = corr)

np.random.seed(train_query_seed)
X_train, train_df, train_true_df, X_est, est_df, est_true_df, X_query, query_df, query_true_df = gen_linear_dgp.gen_train_est_query(n_train = n_train, n_est = n_est, n_query = n_query)

In [ ]:
n_iter = 1
confint_df_list = []
for iter in range(n_iter):
    print(iter)
    seed = 42 * iter
    np.random.seed(seed)

    ## resample X and T
    df_iter, df_true_iter = gen_linear_dgp.gen_resampled_dataset(X_est)

    X_est_iter = X_est.copy()
    T_est_iter = df_iter['T'].values
    Y_est_iter = df_iter['Y'].values

    ## estimate confidence intervals on this iteration
    knn_match_iter = knn_match(prop_score = gen_linear_dgp.prop_score, outcome_reg = gen_linear_dgp.outcome_reg, ymin = gen_linear_dgp.ymin, ymax = gen_linear_dgp.ymax)
    knn_match_iter.learn_projection(X_train, train_df['T'].values, train_df['Y'].values, algorithm = 'true_coef', args = {'coef' : np.array([1,1])})# args = {'coef' : gen_linear_dgp.coef})
    # knn_match_iter.learn_projection(X_train, train_df['T'].values, train_df['Y'].values, algorithm = 'true_prog')
    knn_match_iter.fit(X_est_iter, T_est_iter, Y_est_iter, k = k)

    lb_iter, ub_iter, bias_iter = knn_match_iter.est_cate_conf_int(X_query, alpha = 0.05, return_bias = True)
    cate_mg = knn_match_iter.est_cate_mg_or(X_query)
    cate_est = knn_match_iter.est_cate(X_query)

    true_df_iter = query_true_df.copy()
    true_df_iter['lb'] = lb_iter
    true_df_iter['ub'] = ub_iter
    true_df_iter['bias'] = bias_iter
    true_df_iter['seed'] = seed
    true_df_iter['cate_mg'] = cate_mg
    true_df_iter['cate_est'] = cate_est

    confint_df_list.append(true_df_iter)

In [ ]:
# indices = knn_match_iter.find_nn(X_query)
# # get treatment and outcome of nearest neighbors
# mu_nn_T = knn_match_iter.mu_x_T[indices].mean(axis = 1)
# mu_nn_C = knn_match_iter.mu_x_C[indices].mean(axis = 1)
cate_or = (knn_match_iter.outcome_reg(X_query, T = 1) - knn_match_iter.outcome_reg(X_query, T = 0)).flatten()

In [ ]:
((true_df_iter['cate_true'] - true_df_iter['cate_est'])**2).mean()

In [ ]:
## why is the interval width so tight for SAVI?
## the width for normal approx is 0.03... but the width for SAVI is 0.023...
pd.concat([x.assign(id = range(x.shape[0])).assign(coverage = lambda x: (x.lb <= x.cate_true) * (x.cate_true <= x.ub)).assign(length = lambda x: (x.ub - x.lb)/(gen_linear_dgp.ymax - gen_linear_dgp.ymin)) for x in confint_df_list], axis = 0).groupby('id')[['coverage', 'length']].mean().mean()

In [ ]:
confint_df_full = pd.concat([x.assign(id = range(x.shape[0])).assign(coverage = lambda x: (x.lb <= x.cate_true) * (x.cate_true <= x.ub)).assign(length = lambda x: (x.ub - x.lb)/(gen_linear_dgp.ymax - gen_linear_dgp.ymin)) for x in confint_df_list], axis = 0)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
x_gb_df = confint_df_full.groupby('id')[['X_0', 'X_1', 'T', 'coverage', 'length']].mean()
sns.scatterplot(x = 'X_0', y = 'X_1', hue = (gen_linear_dgp.prop_score(df_iter.drop(['Y', 'T'], axis = 1))).flatten(), data = df_iter)
plt.title('Propensity score')
plt.show()
sns.scatterplot(x = 'X_0', y = 'X_1', hue = 'coverage', data = x_gb_df)
plt.show()
sns.scatterplot(x = 'X_0', y = 'X_1', hue = (gen_linear_dgp.outcome_reg(df_iter.drop(['Y', 'T'], axis = 1), T = 1) - gen_linear_dgp.outcome_reg(df_iter.drop(['Y', 'T'], axis = 1), T = 0)).flatten(), data = df_iter)
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

indices = knn_match_iter.nbrs.kneighbors(X_query, return_distance = False)
query_point = X_query.index[20]
nn_idxs = indices[query_point, ]
# Extract the points that meet the condition
df_iter['in_mg'] = 0
df_iter.loc[nn_idxs, 'in_mg'] = 1
points = df_iter.loc[nn_idxs, ['X_0', 'X_1']].values

# Compute the convex hull
hull = ConvexHull(points)

# Plot the points
sns.scatterplot(x='X_0', y='X_1', hue = 'in_mg', data=df_iter)

# Plot the convex hull
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
star = X_query.loc[query_point, ['X_0', 'X_1']].values
plt.plot(star[0], star[1], 'ro', markersize = 18)
plt.show()


In [ ]:
confint_df_full['lb_wo_bias'] = confint_df_full['lb'] - confint_df_full['bias']
confint_df_full['ub_wo_bias'] = confint_df_full['ub'] - confint_df_full['bias']

confint_df_full['lb_w_2bias'] = confint_df_full['lb'] + confint_df_full['bias']
confint_df_full['ub_w_2bias'] = confint_df_full['ub'] + confint_df_full['bias']

In [ ]:
# length = 0.27
# length = 0.625
confint_df_full['coverage_wo_bias'] = (confint_df_full['lb_wo_bias'] <= confint_df_full['cate_true']) * (confint_df_full['cate_true'] <= confint_df_full['ub_wo_bias'])
confint_df_full['interval_width_wo_bias'] = confint_df_full['ub_wo_bias'] - confint_df_full['lb_wo_bias']

confint_df_full['coverage_mg_wo_bias'] = (confint_df_full['lb_wo_bias'] <= confint_df_full['cate_mg']) * (confint_df_full['cate_mg'] <= confint_df_full['ub_wo_bias'])
confint_df_full['interval_width_mg_wo_bias'] = confint_df_full['ub_wo_bias'] - confint_df_full['lb_wo_bias']

confint_df_full['coverage_w_bias'] = (confint_df_full['lb'] <= confint_df_full['cate_true']) * (confint_df_full['cate_true'] <= confint_df_full['ub'])
confint_df_full['interval_width_w_bias'] = confint_df_full['ub'] - confint_df_full['lb']

confint_df_full['coverage_mg_w_bias'] = (confint_df_full['lb'] <= confint_df_full['cate_mg']) * (confint_df_full['cate_mg'] <= confint_df_full['ub'])
confint_df_full['interval_width_mg_w_bias'] = confint_df_full['ub'] - confint_df_full['lb']

confint_df_full['coverage_w_2bias'] = (confint_df_full['lb_w_2bias'] <= confint_df_full['cate_true']) * (confint_df_full['cate_true'] <= confint_df_full['ub_w_2bias'])
confint_df_full['interval_width_w_2bias'] = confint_df_full['ub_w_2bias'] - confint_df_full['lb_w_2bias']

confint_df_full['coverage_mg_w_2bias'] = (confint_df_full['lb_w_2bias'] <= confint_df_full['cate_mg']) * (confint_df_full['cate_mg'] <= confint_df_full['ub_w_2bias'])
confint_df_full['interval_width_mg_w_2bias'] = confint_df_full['ub_w_2bias'] - confint_df_full['lb_w_2bias']

In [ ]:
confint_df_full.groupby('id')[['coverage_wo_bias', 'coverage_w_bias', 'coverage_w_2bias', 'coverage_mg_wo_bias', 'coverage_mg_w_bias', 'coverage_mg_w_2bias']].mean().mean()

In [ ]:
n_train = 1000
n_est = 1000
n_query = 100
train_query_seed = 42
xmin = -5
xmax = 2
n_imp = 5
n_unimp = 0
heteroskedastic = 0

k = 2 * int(np.sqrt(n_est))

# gen_linear_dgp = dgp_lihua(xmin = xmin, xmax = xmax, n_imp = n_imp, n_unimp=n_unimp, heteroskedastic=heteroskedastic, corr = False)
gen_linear_dgp = dgp_linear(xmin = xmin, xmax = xmax, n_imp = n_imp, n_unimp = n_unimp, heteroskedastic=heteroskedastic)

np.random.seed(train_query_seed)
X_train, train_df, train_true_df, X_est, est_df, est_true_df, X_query, query_df, query_true_df = gen_linear_dgp.gen_train_est_query(n_train = n_train, n_est = n_est, n_query = n_query)
T_train = train_df['T'].values
Y_train = train_df['Y'].values

confint_df_list = []
for k_constant in range(1, 20, 3):
    print(k_constant)
    k = k_constant * int(np.sqrt(n_est))
    if k > n_est:
        k = n_est
    seed = 42
    np.random.seed(seed)

    ## resample X and T
    df_iter, df_true_iter = gen_linear_dgp.gen_resampled_dataset(X_est)

    X_est_iter = X_est.copy()
    T_est_iter = df_iter['T'].values
    Y_est_iter = df_iter['Y'].values

    ## estimate confidence intervals on this iteration
    class or_class:
        def __init__(self, or_estimator, or_args = {}):
            self.or_T = or_estimator(**or_args)
            self.or_C = or_estimator(**or_args)
        def fit(self, X_train, T_train, Y_train):
            X_train = np.array(X_train)
            self.or_T.fit(X_train[T_train == 1, ], Y_train[T_train == 1])
            self.or_C.fit(X_train[T_train == 0, ], Y_train[T_train == 0])
        def predict(self, X_query, T):
            if T == 1:
                return self.or_T.predict(X_query).reshape(-1,1)
            elif T == 0:
                return self.or_C.predict(X_query).reshape(-1,1)
            else:
                raise ValueError('T must be 0/1. Uncrecognized T:', T)
        
    outcome_reg = or_class(RandomForestRegressor)
    outcome_reg.fit(X_train, T_train, Y_train)

    # prop_score None defaults to using NNs...
    knn_match_iter = knn_match(prop_score = None, outcome_reg = outcome_reg.predict, ymin = gen_linear_dgp.ymin, ymax = gen_linear_dgp.ymax, bias_correction = True)

    ## fit projection on training set
    knn_match_iter.learn_projection(X_train, T_train, Y_train, algorithm = 'rf_prognostic_score')

    ## estimate intervals on estimation set
    knn_match_iter.fit(X_est_iter, T_est_iter, Y_est_iter, k = k)

    lb_iter, ub_iter = knn_match_iter.est_cate_conf_int(X_query, alpha = 0.05)


    # knn_match_iter = knn_match(prop_score = gen_linear_dgp.prop_score, outcome_reg = gen_linear_dgp.outcome_reg, ymin = gen_linear_dgp.ymin, ymax = gen_linear_dgp.ymax)
    # knn_match_iter = knn_match(prop_score = None, outcome_reg = lambda X, T: np.zeros((X.shape[0], 1)), ymin = gen_linear_dgp.ymin, ymax = gen_linear_dgp.ymax)
    # knn_match_iter.learn_projection(X_train, train_df['T'].values, train_df['Y'].values, algorithm = 'true_coef', args = {'coef' : gen_linear_dgp.coef})
    # knn_match_iter.fit(X_est_iter, T_est_iter, Y_est_iter, k = k)

    # lb_iter, ub_iter = knn_match_iter.est_cate_conf_int(X_query, alpha = 0.05)

    true_df_iter = query_true_df.copy()
    true_df_iter['lb'] = lb_iter
    true_df_iter['ub'] = ub_iter
    true_df_iter['seed'] = seed
    true_df_iter['k'] = k

    confint_df_list.append(true_df_iter)

In [ ]:
pd.concat([x.assign(id = range(x.shape[0])).assign(coverage = lambda x: (x.lb <= x.cate_true) * (x.cate_true <= x.ub)).assign(length = lambda x: (x.ub - x.lb)/(gen_linear_dgp.ymax - gen_linear_dgp.ymin)) for x in confint_df_list], axis = 0).groupby(['id', 'k'])[['coverage', 'length']].mean().reset_index().groupby('k')[['coverage', 'length']].mean()

### IPW ATE Coverage

In [ ]:
n_train = 1000
n_est = 50000
n_query = 1
train_query_seed = 42
xmin = -5
xmax = 5
n_imp = 2
n_unimp = 0
heteroskedastic = 0
corr = False

k = n_est

gen_linear_dgp = dgp_lihua(xmin = xmin, xmax = xmax, n_imp = n_imp, n_unimp=n_unimp, heteroskedastic=heteroskedastic, corr = corr)

np.random.seed(train_query_seed)
X_train, train_df, train_true_df, X_est, est_df, est_true_df, X_query, query_df, query_true_df = gen_linear_dgp.gen_train_est_query(n_train = n_train, n_est = n_est, n_query = n_query)

In [ ]:
n_iter = 100
confint_list = []
coverage = 0
for iter in range(n_iter):
    print(iter)
    seed = 42 * iter
    np.random.seed(seed)

    ## resample X and T
    df_iter, df_true_iter = gen_linear_dgp.gen_resampled_dataset(X_est)

    X_est_iter = X_est.copy()
    T_est_iter = df_iter['T'].values
    Y_est_iter = df_iter['Y'].values

    ## estimate confidence intervals on this iteration
    knn_match_iter = knn_match(prop_score = gen_linear_dgp.prop_score, outcome_reg = lambda X, T : np.zeros(X.shape[0]), ymin = gen_linear_dgp.ymin, ymax = gen_linear_dgp.ymax)
    knn_match_iter.learn_projection(X_train, train_df['T'].values, train_df['Y'].values, algorithm = 'true_coef', args = {'coef' : np.array([1,1])})# args = {'coef' : gen_linear_dgp.coef})
    # knn_match_iter.learn_projection(X_train, train_df['T'].values, train_df['Y'].values, algorithm = 'true_prog')
    knn_match_iter.fit(X_est_iter, T_est_iter, Y_est_iter, k = k)

    lb_iter, ub_iter, bias_iter = knn_match_iter.est_cate_conf_int(X_query, alpha = 0.05, return_bias = True)
    lb_iter = lb_iter - bias_iter
    ub_iter = ub_iter - bias_iter

    cate_mg = knn_match_iter.est_cate_mg_or(X_query)
    cate_est = knn_match_iter.est_cate(X_query)

    ate_true = (gen_linear_dgp.outcome_reg(X_est, T = 1) - gen_linear_dgp.outcome_reg(X_est, T = 0)).mean()

    coverage += (lb_iter <= ate_true) * (ate_true <= ub_iter)
    print('bias', bias_iter)
    print('coverage', coverage/(iter + 1))

0
[0.]
1
[0.]
2
[0.]
3
[0.]
4
[0.]
5
[0.]
6
[0.]
7
[0.]
8
[0.]
9
[0.]
10
[0.]
11
[0.]
12
[0.]
13
[0.]
14
[0.]
15
[0.]
16
[0.]
17
[0.]
18
[0.]
19
[0.]
20
[0.]
21
[0.]
22
[0.]
23
[0.]
24
[0.]
25
[0.]
26
[0.]
27
[0.]
28
[0.]
29
[0.]
30
[0.]
31
[0.]
32
[0.]
33
[0.]
34
[0.]
35
[0.]
36
[0.]
37
[0.]
38
[0.]
39
[0.]
40
[0.]
41
[0.]
42
[0.]
43
[0.]
44
[0.]
45
[0.]
46
[0.]
47
[0.]
48
[0.]
49
[0.]
50
[0.]
51
[0.]
52
[0.]
53
[0.]
54
[0.]
55
[0.]
56
[0.]
57
[0.]
58
[0.]
59
[0.]
60
[0.]
61
[0.]
62
[0.]
63
[0.]
64
[0.]
65
[0.]
66
[0.]
67
[0.]
68
[0.]
69
[0.]
70
[0.]
71
[0.]
72
[0.]
73
[0.]
74
[0.]
75
[0.]
76
[0.]
77
[0.]
78
[0.]
79
[0.]
80
[0.]
81
[0.]
82
[0.]
83
[0.]
84
[0.]
85
[0.]
86
[0.]
87
[0.]
88
[0.]
89
[0.]
90
[0.]
91
[0.]
92
[0.]
93
[0.]
94
[0.]
95
[0.]
96
[0.]
97
[0.]
98
[0.]
99
[0.]


In [ ]:
# lb_iter[0], ub_iter[0]
bias_iter

In [ ]:
coverage